# Informe Recuperos

La finalidad del siguiente informe es dar a conocer ciertas inconsistencias detectadas entre los reportes emitidos por el Sistema Recuperos INVICO. De más está decir que no es posible ni es el objetivo del presente emitir juicio alguno sobre el sistema en sí sino solo de sus reportes siendo estos un output del primero.
Para entender la importancia que reviste el mencionado sistema y sus reportes basta con mencionar que, aproximadamente, el **65% del Activo del INVICO está formado por los saldos de recuperos a cobrar** ($5800 millones) superando ampliamente a otros ítems como saldos disponibles en banco ($1600 millones), obras en ejecución y terminadas entre otros de menor relevancia.
A estos efectos, es necesario contar con información pertinente y confiable elaborada de manera sistémica bajo criterios que se mantenga uniforme a lo largo del tiempo respecto a los **saldos de recuperos a cobrar y las causas de sus variaciones**. 

## Datos iniciales

El único dato que debería modificarse es el relativo al ejercicio sobre el cual se pretende trabajar.

In [ ]:
from invicoctrlpy.recursos.recuperos.recuperos import Recuperos

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('mode.chained_assignment',None)

import itables
itables.options.lengthMenu = [10, 50, 100, -1]
itables.options.maxBytes = 0

EJERCICIO = '2022'
UPDATE_DB = False
recuperos = Recuperos(ejercicio=EJERCICIO, update_db=UPDATE_DB)

## Recuperos a cobrar

### Evolución del saldo

Una de las primeras cuestiones a analizar es la evolución, en el tiempo, del saldo final de Recuperos a Cobrar. El siguiente gráfico permite reflejar esta cuestión y surge del [Informe de Saldos por Barrio](https://gv.invico.gov.ar/App/Recupero/Informes/InformeSaldosPorBarrio.aspx) Probablemente se trate de uno de uno de los reportes más importantes del Sistema Recuperos. El saldo del mismo y su evolución es reflejado fielmente en los Estados Contables de la institución y el Dpto. Recuperos lo respeta, en igual medida, en su informe final de ejercicio. En cuanto a la razonabilidad del saldo final expuesto, no hay forma de emitir juicio sobre el mismo y no queda otra que tomarlo como válido. 


In [ ]:
recuperos.graficar_saldos_barrio()

Del análisis del gráfico expuesto no quedan dudas que el ejercicio 2022 presenta un comportamiento anormal en lo que evolución del saldo se refiere. Un análisis más minucioso de sus causas permitirá dilucidar la cuestión.

### Motivos de variación del saldo

Una primera aproximación a esta cuestión la podemos observar en el [Informe Evolucion De Saldos Por Barrio](https://gv.invico.gov.ar/App/Recupero/Informes/InformeEvolucionDeSaldosPorBarrio.aspx). Similar al [Informe de Saldos por Barrio](https://gv.invico.gov.ar/App/Recupero/Informes/InformeSaldosPorBarrio.aspx) más la adición de dos columnas (amortiza y cambios) que intentan explicar las causas que motivaron la variación en saldos indexando todo por código de barrio. Es aquí donde el **Sistema Recuperos presenta su primera falla**. Si uno procede a realizar la **suma algebraica** de la columnas del informe, **no es posible llegar al saldo final del mismo**. La siguiente tabla permite observar la evolución de la difencia entre el saldo final expuesto y el calculado por suma algebraica. Sin embargo, cabe aclarar que, **para el 2022, la diferencia es poco significativa**.

In [ ]:
control_suma_saldo_barrio_variacion = recuperos.control_suma_saldo_barrio_variacion()
itables.show(control_suma_saldo_barrio_variacion, column_filters="footer", dom="lrtip")

A efectos de profundizar en el análisis de causas el reporte ideal es el [Informe de Evolución Saldos Por Motivos](https://gv.invico.gov.ar/App/Recupero/Informes/InformeEvolucionDeSaldosPorMotivos.aspx). Este reporte no es otra cosa que la descomposición de la columna cambios de lo visto en el cuadro anterior.

In [ ]:
rank_saldo_motivo_actual = recuperos.ranking_saldo_motivos_actual().head()
itables.show(rank_saldo_motivo_actual, column_filters="footer", dom="lrtip")

In [ ]:
mejores_motivos = recuperos.control_motivos_actuales_otros_ejercicio()
itables.show(mejores_motivos, column_filters="footer", dom="lrtip")

**Aquí nuevamente el Sistema Recuperos presenta fallas** y es que el **saldo final de este reporte no coincide con lo visto en los anteriores reportes**. Como puede observarse en el siguiente gráfico, las diferencias son más evidentes en los ejercicios 2019 y 2020 (idéntico a lo visto en el cuadro anterior) y alcanza el valor de $15 millones para el ejercicio 2022.

In [ ]:
recuperos.graficar_dif_saldo_final_reportes()

## Evolución de saldos por barrio

In [ ]:
control_suma_saldo_barrio_variacion = recuperos.control_suma_saldo_barrio_variacion()
itables.show(control_suma_saldo_barrio_variacion, column_filters="footer", dom="lrtip")

## EXTRA:

### Excel

In [ ]:
# with pd.ExcelWriter('control_recursos.xlsx') as writer:
#     control_mes_grupo.to_excel(writer, sheet_name='control_mes', index=False)
#     control_mes_gpo_cta_cte.to_excel(writer, sheet_name='control_mes_cta_cte', index=False)
#     siif.to_excel(writer, sheet_name='siif', index=False)
#     sscc.to_excel(writer, sheet_name='sscc', index=False)